# 생활인구 데이터 홈페이지
https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do

In [1]:
from selenium import webdriver
import time
import folium
import os
import json
import pandas as pd
import numpy as np
import datetime

In [2]:
data_path = '/home/zimin/datasets/seoul/'
geo_path = data_path + 'skorea_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str['features'] = [i for i in geo_str['features'] if i['properties']['code'][:2] == '11']

In [3]:
pd.set_option('max_colwidth',500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [8]:
analysis = datetime.datetime(2019,11, 25)
week = {0 : '월요일', 1 : '화요일', 2 : '수요일', 3 : '목요일', 4 : '금요일', 5 : '토요일', 6 : '일요일'}
weekday = week[analysis.weekday()]
date = str(analysis.year) + str(analysis.month) + str(analysis.day) 

In [9]:
weekday

'월요일'

In [12]:
local = pd.read_csv(os.path.join(data_path, 'pre_local_PEOPLE_' + date + '.csv'), index_col=[0])

In [13]:
local

,기준일ID,시간대구분,행정동코드,행정동이름,총생활인구수,110,115,120,125,130,135,140,145,150,155,160,165,170,210,215,220,225,230,235,240,245,250,255,260,265,270
0,20191125,0,1101072,청운효자동,585.0,22,16,28,22,17,17,21,26,16,20,10,10,19,11,18,16,16,19,17,26,38,22,28,12,15,37
1,20191125,0,1101072,청운효자동,830.0,10,18,27,33,21,39,32,47,26,21,19,17,43,12,20,25,24,33,36,31,44,43,34,31,19,74
2,20191125,0,1101072,청운효자동,47.0,*,*,*,*,*,4,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,4
3,20191125,0,1101072,청운효자동,463.0,9,9,13,10,13,16,20,22,17,13,14,7,17,13,17,14,18,15,26,27,28,17,15,15,8,20
4,20191125,0,1101072,청운효자동,161.0,4,4,8,5,*,7,6,6,4,6,*,*,6,*,4,5,*,6,4,6,10,4,8,*,*,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459644,20191125,23,1125071,둔촌2동,968.0,10,22,18,28,34,39,39,36,22,35,30,23,51,24,32,29,30,28,47,44,43,44,39,34,30,78
459645,20191125,23,1125071,둔촌2동,240.0,*,5,4,7,8,10,9,9,5,9,7,5,12,6,8,7,7,6,12,11,10,11,8,8,7,19
459646,20191125,23,1125071,둔촌2동,1256.0,20,28,17,32,41,48,64,38,36,40,43,45,82,25,30,26,32,38,49,46,60,38,73,49,48,109
459647,20191125,23,1125071,둔촌2동,230.0,10,6,7,7,6,5,8,8,4,10,7,5,7,6,5,7,4,6,7,7,12,7,12,7,8,6


In [15]:
when = 3
who = ['140','240']
whose = who[0][1:] + '~' + str(eval(who[-1][1:] )+5)

In [16]:
date, weekday, when, whose

('20191125', '월요일', 3, '40~45')

In [17]:
check = local[(local['시간대구분'] == when)][['행정동이름','행정동코드'] + who]
check[who] = check[who].replace('*',0)
check[who] = check[who].astype(int)
check['sum'] = check[who].sum(axis=1)
check['행정동코드'] = check['행정동코드'].apply(str)

In [18]:
check.sort_values(by='sum',ascending=False).head(10)

,행정동이름,행정동코드,140,240,sum
75200,가락1동,1124066,1494,1127,2621
57811,신당동,1102069,912,881,1793
76119,천호1동,1125061,848,854,1702
75408,위례동,1124082,448,478,926
58937,성수2가3동,1104068,450,433,883
66796,목1동,1115051,394,454,848
65358,진관동,1112071,362,417,779
68512,발산1동,1116067,355,351,706
70182,여의동,1119054,410,277,687
75335,문정2동,1124069,313,341,654


In [70]:
fn= date + weekday + ' ' +str(when)+ '시 ' + whose + '세' + '.html'

seoul_map = folium.Map(location =[37.5502, 126.982], zoom_start=11, tiles = 'cartodbpositron')
seoul_map.choropleth(geo_data = geo_str,
                    data = check,
                    columns = ['행정동코드', 'sum'],
                    fill_color = 'Reds',
                    bins = 9,
                    key_on = 'properties.code',
                    highlight = True,
                    fill_opacity = 0.7, line_opacity = 1,
                    legend_name = 'good_boy')

title_html = '''
             <h3 align="center" style="font-size:20px"><b>{}</b></h3>
             '''.format(fn[:-5])
seoul_map.get_root().html.add_child(folium.Element(title_html))

delay=1
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd() + '/html_data',mapfile=fn)
seoul_map.save('./html_data/' + fn)
browser = webdriver.Chrome('/usr/bin/chromedriver')
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('./screenshot/' + fn[:-5] + '.png')
browser.quit()